In [3]:
import os

import bs4 as bs
import requests

import sqlite3
import pandas as pd
from pandas import DataFrame,Series

* Some starter notes

In [ ]:
# Parse a link
# link = 'https://www.r-bloggers.com/getting-started-with-postgresql-in-r/'

# Parse with requests library
# r = requests.get(link)
# print(r.content)

# Alternatively access the webpage using selenium library
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.get(link)

# soup the link
# soup = bs.BeautifulSoup(r.content,'lxml')

# print the webpage content
# soup
# print(soup.prettify())
# for x in soup.find_all(['h1','h2','h3','p']):
#     print(x.text)
# soup.title.text
# for x in soup.find_all('a'):
#     print(x.get('href')) 
# results = soup.find_all("span",{"class":class_to_try})
# driver.find_element_by_xpath('//*[@id="react-root"]/div[1]/main/div/div/form/div/div[3]').click()


* custom function

In [2]:
def parse(link):
    
    try:
        r = requests.get(link)
        soup = bs.BeautifulSoup(r.content,'lxml')
        
        # get title
        title = soup.title.text.strip()
        
        # get body
        pre = []
        for x in soup.find_all(['h1','h2','h3','p']):
            pre.append(x.text)
            body = ' '.join(pre).replace("\n"," ").replace("\r"," ").replace("\xa0"," ")
        
        # get list with hrefs
#         hreflist = []
#         for x in soup.find_all('a'):
#             hreflist.append(x.get('href'))

        return [title,body] #,hreflist
    
    except Exception as e:
        print(e)

In [3]:
class my_db():
    def __init__(self,tablename,dbname):
        self.tablename = tablename
        self.dbname = dbname
        self.db = sqlite3.connect(self.dbname)
        
    def commit(self,df):
        df.to_sql(self.tablename,self.db,if_exists="append",index=False)
        
    def send_q(self,query_string):
        self.db.execute(query_string)
        
    def q(self,query_string):
        cur = self.db.cursor()
        cur.execute(query_string)
        return cur.fetchall()
    
    def get_ids(self):
        self.idlist = [x[0] for x in db.q("select distinct link from marks;")]
        return self.idlist
    
    def close(self):
        self.db.close()

In [4]:
def commit_chunk(scrape_list,db_obj,chunk_size=10,verbose=True):
    # initialize
    parsed_list = []
    title_list = []
    body_list  = []
    j = 1

    for i,y in enumerate(scrape_list,start=1):
        # print status
        if verbose:
            print(i)
            print(f"parsing {y}")
            print()
        # parse
        try :
            title, body = parse(y)
        except Exception as e:
            print(e)
        # hold in memory
        parsed_list.append(y)
        title_list.append(title)
        body_list.append(body)
        # commit to database
        if any([len(title_list)==chunk_size,i==len(scrape_list)]):
            # turn lists into a dict
            temp_dc = {'link':parsed_list,
                       'title':title_list,
                       'body':body_list}
            # turn dict into a df
            temp_df = DataFrame(temp_dc)
            # commit the df
            db_obj.commit(temp_df)
            # erase memory
            parsed_list = []
            title_list = []
            body_list  = []
            del temp_df
            del temp_dc
            # print progress
            if verbose:
                print(f"commit {j} completed")
                print(50*'#=')
            j+=1

### Parse some bookmarks

* html bookmarks input

In [6]:
# input html file containing bookmarks to scrape
with open('bookmarks.html', encoding="utf8") as html:
    soup=bs.BeautifulSoup(html, 'lxml')

scrape_list = []
for x in soup.find_all('a'):
    scrape_list.append(x.get('href'))

* workflow

In [16]:
db = my_db("marks","bookmarks.sqlite")

In [19]:
# if table already exists
db.q("select distinct title from marks;")[:5]
len(db.get_ids())
# db.send_q("drop table if exists marks;")

# keep links that do not exist already
scrape_list_new = [x for x in scrape_list if x not in db.get_ids()]

* parse and commit to the db

In [ ]:
commit_chunk(scrape_list_new,db,chunk_size=10,verbose=True)

## Incorporate more data
### _Medium_ data from kaggle
* https://www.kaggle.com/hsankesara/medium-articles

In [134]:
med = pd.read_csv("./medium/articles.csv")
medium = med[["link","title","text"]].drop(med.index[23]).drop(med.index[307]).reset_index().drop("index",axis=1).rename(columns = {"text":"body"})

In [ ]:
conn = sqlite3.connect("bookmarks.sqlite")
medium.to_sql("med",conn,index=False)

* merge all in one table

In [137]:
cur = conn.cursor()
cur.execute("""
            CREATE TABLE full AS 
            SELECT * FROM marks
            UNION
            SELECT * FROM med;
            """)

### KDNuggets
* Parse a local directory with html files

In [1]:
def parse_soup(soup):
    
    try:
        title = soup.title.text.strip()
        pre = []
        for x in soup.find_all(['h1','h2','h3','p']):
            pre.append(x.text)
            body = ' '.join(pre).replace("\n"," ").replace("\r"," ").replace("\xa0"," ")
        return [title,body]
    
    except Exception as e:
        pass

In [34]:
title_list = []
body_list = []

setwd = "\\".join([os.getcwd(),"projects","WebScraping","kdnuggets","clean"])
os.chdir(setwd)

for entry in os.listdir(os.getcwd()):
    with open(entry, encoding="utf8") as html:
        try:
            soup=bs.BeautifulSoup(html, 'lxml')
            title, body = parse_soup(soup)

            title_list.append(title)
            body_list.append(body)
        except Exception as e:
            pass

In [37]:
palantas = DataFrame({'title':title_list,
                     'body':body_list})

palantas['link'] = 'na'

boobos = palantas[['link','title','body']]

In [65]:
conn = sqlite3.connect("C:/Users/takis/Desktop/PyR/projects/WebScraping/bookmarks.sqlite")

In [66]:
boobos.to_sql("nuggs",conn,if_exists="replace",index=False)

In [67]:
cur = conn.cursor()
cur.execute("""
            CREATE TABLE full2 AS 
            SELECT * FROM full
            UNION
            SELECT * FROM nuggs;
            """)

### _AnalyticsVidhya_
* parse a local directory with html files

In [ ]:
title_list = []
body_list = []

where_to_look = r'C:\Users\takis\Desktop\PyR\projects\WebScraping\analyticsvidhya\clean'

for entry in os.listdir(where_to_look):
    fullnamepath = "\\".join([where_to_look,entry])
    if not os.path.isdir(fullnamepath):
        with open(fullnamepath, encoding="utf8") as html:
            try:
                soup=bs.BeautifulSoup(html, 'lxml')
                title, body = parse_soup(soup)

                title_list.append(title)
                body_list.append(body)
            except Exception as e:
                pass

In [10]:
d0 = DataFrame({'title':title_list,
                     'body':body_list})

d0['link'] = 'na'

d1 = d0[['link','title','body']]

In [11]:
conn = sqlite3.connect("C:/Users/takis/Desktop/PyR/projects/WebScraping/bookmarks.sqlite")

In [12]:
d1.to_sql("lytics",conn,if_exists="replace",index=False)

### Notebook

* check existence before commiting
* add the word count for every article
* parse list with href?

In [ ]:
# loop to parse links
title_list = []
body_list  = []
parsed_list = []
j = 1
# href_list  = []

for y in scrape_list[0:50]:
#     title, body, hreflist = parse(y)
    title, body = parse(y)

    parsed_list.append(y)
    title_list.append(title)
    body_list.append(body)
#     href_list.append(hreflist)
#     print(title)

    if len(title_list)==10:
        print(len(title_list))
        to_db("marks",'bookmarks.sqlite',title_list,body_list)
        title_list = []
        body_list  = []
        print(f"this is the {j} commit to the db...")
        j+=1

In [ ]:
title, body = parse(scrape_list[0])

In [ ]:
title_list = []
body_list  = []
parsed_list = []
j = 1
# href_list  = []

for y in scrape_list[0:3]:
#     title, body, hreflist = parse(y)
    title, body = parse(y)

    parsed_list.append(y)
    title_list.append(title)
    body_list.append(body)